TESTE - Extraindo texto de uma imagem usando tesseract

In [1]:
import os
import re
from PIL import Image
import pytesseract

# Caminho do executável do Tesseract (necessário no Windows)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

imagem = Image.open("C:/Users/rafaf/Documents/Filigrana/ocr-catalogo/imagens/005.14057.jpg")
texto = pytesseract.image_to_string(imagem, lang="eng")

print("Texto extraído:", texto)


Texto extraído: 0051405701
R$ 38,17




EXECUÇÃO - Extraindo texto de todas as imagens de uma pasta usando tesseract e tratando elas para um .csv

In [ ]:
import os
import re
from PIL import Image
import pytesseract
import csv

# 1. colocar imagens de uma categoria (ex: Brincos) na pasta imagens 
# 2. Mudar categoria (nome da pasta)
# 3. Trocar o nome do Nmodelo

# Configuração do caminho do Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Pasta das imagens
pasta = "./imagens"
extensoes = ('.png', '.jpg', 'jpeg')
resultados = []

# Função para processar o texto extraído e estruturar os dados
def processar_texto(texto, arquivo):
    linhas = texto.strip().split("\n")  # Quebrar por linha e remover espaços extras
    linhas = [linha.strip() for linha in linhas if linha.strip()]  # Remover linhas vazias

    if len(linhas) >= 2:
        nomeIDcompleto = linhas[0]  # Primeira linha nomeID
        preco = linhas[1] # Segunda linha como preço
    else:
        nomeIDcompleto = "Desconhecido"
        preco = "0.00"

    # Limpar o preço (removendo caracteres indesejados e deixando apenas números e vírgula)
    preco = preco.replace("R$", "").strip()  # Remove "R$"
    preco = re.sub(r"[^0-9,]", "", preco)  # Mantém apenas números e vírgula   

    # Se o preço estiver vazio, definir como "0,00"
    if not preco:
        preco = "0,00" 

    # Separar nomeID e tamanho se houver um ponto no nomeID
    if "." in nomeIDcompleto:
        partes = nomeIDcompleto.split(".")
        nomeID = partes[0]  # Parte antes do ponto é o nomeID
        tamanho = partes[1] if partes[1].isdigit() else ""  # Parte depois do ponto é o tamanho se for número
    else:
        nomeID = nomeIDcompleto
        tamanho = ""    

    # Definir Ntipo com base no final de nomeID
    if nomeID.endswith("01"):
        Ntipo = ""
    elif nomeID.endswith("66"):
        Ntipo = "DISPONIVEL NA PRATA 925"
    elif nomeID.endswith("73"):
        Ntipo = "DISPONIVEL NO RODIO"
    elif nomeID.endswith("45"):
        Ntipo = ""
    elif nomeID.endswith("65"):
        Ntipo = ""
    else:
        Ntipo = "N/A"  # Caso o código não termine com os valores esperados

    # Criar o nome completo
    Nmodelo = "GARGANTILHA"
    nome = f"{Nmodelo} {Ntipo}".strip()

    sku = os.path.splitext(arquivo)[0]    

    return {
        "Nome": nome,
        "Categoria": "24 07 25",
        "Variacao": tamanho,
        "SKU": sku,
        "Preco": preco,
        "codigo": nomeID
    }

# Processar cada imagem na pasta
for arquivo in os.listdir(pasta):
    if arquivo.lower().endswith(extensoes):
        img = os.path.join(pasta, arquivo)
        texto = pytesseract.image_to_string(img, lang="eng")

        dados_formatados = processar_texto(texto, arquivo)
        resultados.append(dados_formatados)

# Salvar os resultados em CSV
csv_path = "resultadosT.csv"
with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Nome", "Categoria", "SKU", "Variacao", "Preco", "codigo"], delimiter=';')
    writer.writeheader()
    writer.writerows(resultados)

print(f"Resultados salvos em: {csv_path}")


Resultados salvos em: resultadosT.csv


TESTE - Extraindo texto de uma imagem usando easyocr

In [33]:

import easyocr


# Inicializa o leitor do EasyOCR com o idioma desejado
reader = easyocr.Reader(['en'],gpu=True)  # 'en' para inglês

# Caminho para a imagem
imagem_caminho = "./imagens/005.11449.jpg"

# Processar a imagem
texto = reader.readtext(imagem_caminho, detail=0)  # detail=0 retorna apenas o texto

# Exibir o texto extraído
print("Texto extraído:", " ".join(texto))


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Texto extraído: 0051144901 RS 39,73


EXECUÇÃO - Extraindo texto de todas as imagens de uma pasta usando easyocr e tratando elas para um .csv

In [1]:
import os
import re
import csv
import easyocr

# Inicializa o leitor do EasyOCR
reader = easyocr.Reader(['en'], gpu=True)  # 'en' para inglês

# Pasta das imagens
pasta = "./imagens"
extensoes = ('.png', '.jpg', '.jpeg')
resultados = []

# Função para processar o texto extraído
def processar_texto(texto, arquivo):
    # Remover espaços extras e garantir que seja apenas um espaço entre os elementos
    texto = " ".join(texto.split())

    # Expressão regular para encontrar o primeiro conjunto de números (código)
    match_codigo = re.search(r"\b\d+\b", texto)
    codigo = match_codigo.group(0) if match_codigo else "Desconhecido"

    # Expressão regular para encontrar o último número no formato de preço (ex: 39,73)
    match_preco = re.findall(r"\d{1,3}(?:,\d{2})?", texto)  # Busca números que podem ter vírgula
    preco = match_preco[-1] if match_preco else "0,00"

    # Criar nome e categoria baseados no código
    if codigo.endswith("01"):
        Ntipo = ""
    elif codigo.endswith("66"):
        Ntipo = "DISPONIVEL NA PRATA 925"
    elif codigo.endswith("73"):
        Ntipo = "DISPONIVEL NO RODIO"
    elif codigo.endswith("45"):
        Ntipo = ""
    elif codigo.endswith("65"):
        Ntipo = ""
    else:
        Ntipo = "N/A"

    nome = f"PULSEIRA {Ntipo}".strip()
    sku = os.path.splitext(arquivo)[0]

    return {
        "Nome": nome,
        "Categoria": "03 25",
        "Variacao": "",
        "SKU": sku,
        "Preco": preco,
        "codigo": codigo
    }

# Processar cada imagem na pasta
for arquivo in os.listdir(pasta):
    if arquivo.lower().endswith(extensoes):
        img = os.path.join(pasta, arquivo)
        
        # Extrai texto com EasyOCR
        texto_easyocr = " ".join(reader.readtext(img, detail=0))

        # Processa os dados extraídos
        dados_formatados = processar_texto(texto_easyocr, arquivo)
        resultados.append(dados_formatados)

# Salvar os resultados em CSV
csv_path = "resultadosE.csv"
with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Nome", "Categoria", "SKU", "Variacao", "Preco", "codigo"], delimiter=';')
    writer.writeheader()
    writer.writerows(resultados)

print(f"Resultados salvos em: {csv_path}")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Resultados salvos em: resultadosE.csv
